<img src="http://etrago.readthedocs.io/en/latest/_images/etrago_logo.png" alt="HSF" height="200" width="200"  align="center" >



# <center> Abschlussworkshop eGon 2023 </center>

##  <center> Session eTraGo: electric Transmission Grid optimization  <center>



# Agenda <a class="anchor" id="toc"></a>


* [Important links](#links)
* [Installation](#install)
* [General procedure](#general)
* [Use case: eGon2035 scenario optimization](#useCase)
* [Outputs](#outputs)


# Important links <a class="anchor" id="links"></a>

* __[eTraGo Source Code](https://github.com/openego/eTraGo)__
* __[eTraGo Documentation](http://etrago.readthedocs.io/)__


# Installation <a class="anchor" id="install"></a>
The current eTraGo version and the python package jupyter are required to use this notebook. Install these with

`pip install eTraGo`

`pip install jupyterlab`

# General procedure<a class="anchor" id="general"></a>

1. Choose parameter settings (args) in appl.py.
2. Run appl.py or etrago(args).
3. Evaluate results.


# Use case: eGon2035 scenario optimization<a class="anchor" id="useCase"></a>


## Minimal example

1. Where and how much AC/DC network transmission capacity is required?
2. Optimal distribution of different storage technologies
3. System costs

In [ ]:
%%capture
# enable jupyter interactive plotting
%matplotlib notebook
from ipywidgets import *
%matplotlib inline

#import plotting function
from etrago.tools.plot import plot_carrier

### Import required general and eTraGo specific python packages

In [ ]:
%%capture
import datetime
import os
import os.path

import numpy as np

__copyright__ = (
    "Flensburg University of Applied Sciences, "
    "Europa-Universität Flensburg, Centre for Sustainable Energy Systems, "
    "DLR-Institute for Networked Energy Systems"
)
__license__ = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__author__ = (
    "ulfmueller, lukasol, wolfbunke, mariusves, s3pp, ClaraBuettner, "
    "CarlosEpia, KathiEsterl, fwitte, gnn, pieterhexen, AmeliaNadal"
)

if "READTHEDOCS" not in os.environ:
    # Sphinx does not run this code.
    # Do not import internal packages directly

    from etrago import Etrago


### Define parameters to run eTraGo

In [ ]:
args = {
    # Setup and Configuration:
    "db": "egon-data",  # database session
    "gridversion": None,  # None for model_draft or Version number
    "method": {  # Choose method and settings for optimization
        "type": "lopf",  # type of optimization, currently only 'lopf'
        "n_iter": 1,  # abort criterion of iterative optimization, 'n_iter' or 'threshold'
        "pyomo": True,  # set if pyomo is used for model building
    },
    "pf_post_lopf": {
        "active": True,  # choose if perform a pf after lopf
        "add_foreign_lopf": True,  # keep results of lopf for foreign DC-links
        "q_allocation": "p_nom",  # allocate reactive power via 'p_nom' or 'p'
    },
    "start_snapshot": 1,
    "end_snapshot": 15,
    "solver": "gurobi",  # glpk, cplex or gurobi
    "solver_options": {
        "BarConvTol": 1.0e-5,
        "FeasibilityTol": 1.0e-5,
        "method": 2,
        "crossover": 0,
        "logFile": "solver_etrago.log",
        "threads": 7,
    },
    "model_formulation": "kirchhoff",  # angles or kirchhoff
    "scn_name": "eGon2035",  # scenario: eGon2035 or eGon100RE
    # Scenario variations:
    "scn_extension": None,  # None or array of extension scenarios
    "scn_decommissioning": None,  # None or decommissioning scenario
    # Export options:
    "lpfile": False,  # save pyomo's lp file: False or /path/to/lpfile.lp
    "csv_export": "results",  # save results as csv: False or /path/tofolder
    # Settings:
    "extendable": {
        "extendable_components": [
            "as_in_db"
        ],  # Array of components to optimize
        "upper_bounds_grid": {  # Set upper bounds for grid expansion
            # lines in Germany
            "grid_max_D": None,  # relative to existing capacity
            "grid_max_abs_D": {  # absolute capacity per voltage level
                "380": {"i": 1020, "wires": 4, "circuits": 4},
                "220": {"i": 1020, "wires": 4, "circuits": 4},
                "110": {"i": 1020, "wires": 4, "circuits": 2},
                "dc": 0,
            },
            # border crossing lines
            "grid_max_foreign": 4,  # relative to existing capacity
            "grid_max_abs_foreign": None,  # absolute capacity per voltage level
        },
    },
    "generator_noise": 789456,  # apply generator noise, False or seed number
    "extra_functionality": {},  # Choose function name or {}
    # Spatial Complexity:
    "network_clustering_ehv": False,  # clustering of HV buses to EHV buses
    "network_clustering": {
        "active": True,  # choose if clustering is activated
        "method": "kmedoids-dijkstra",  # choose clustering method: kmeans or kmedoids-dijkstra
        "n_clusters_AC": 60,  # total number of resulting AC nodes (DE+foreign)
        "cluster_foreign_AC": False,  # take foreign AC buses into account, True or False
        "method_gas": "kmedoids-dijkstra",  # choose clustering method: kmeans or kmedoids-dijkstra
        "n_clusters_gas": 17,  # total number of resulting CH4 nodes (DE+foreign)
        "cluster_foreign_gas": False,  # take foreign CH4 buses into account, True or False
        "k_elec_busmap": False,  # False or path/to/busmap.csv
        "k_gas_busmap": False,  # False or path/to/ch4_busmap.csv
        "bus_weight_tocsv": None,  # None or path/to/bus_weight.csv
        "bus_weight_fromcsv": None,  # None or path/to/bus_weight.csv
        "gas_weight_tocsv": None,  # None or path/to/gas_bus_weight.csv
        "gas_weight_fromcsv": None,  # None or path/to/gas_bus_weight.csv
        "line_length_factor": 1,  # Factor to multiply distance between new buses for new line lengths
        "remove_stubs": False,  # remove stubs bevore kmeans clustering
        "use_reduced_coordinates": False,  # If True, do not average cluster coordinates
        "random_state": 42,  # random state for replicability of clustering results
        "n_init": 10,  # affects clustering algorithm, only change when neccesary
        "max_iter": 100,  # affects clustering algorithm, only change when neccesary
        "tol": 1e-6,  # affects clustering algorithm, only change when neccesary
        "CPU_cores": 7,  # number of cores used during clustering, "max" for all cores available.
    },
    "sector_coupled_clustering": {
        "active": True,  # choose if clustering is activated
        "carrier_data": {  # select carriers affected by sector coupling
            "central_heat": {
                "base": ["CH4", "AC"],
                "strategy": "simultaneous",  # select strategy to cluster other sectors
            },
        },
    },
    "disaggregation": None,  # None, 'mini' or 'uniform'
    # Temporal Complexity:
    "snapshot_clustering": {
        "active": False,  # choose if clustering is activated
        "method": "segmentation",  # 'typical_periods' or 'segmentation'
        "extreme_periods": None,  # consideration of extreme timesteps; e.g. 'append'
        "how": "daily",  # type of period - only relevant for 'typical_periods'
        "storage_constraints": "soc_constraints",  # additional constraints for storages  - only relevant for 'typical_periods'
        "n_clusters": 5,  #  number of periods - only relevant for 'typical_periods'
        "n_segments": 5,  # number of segments - only relevant for segmentation
    },
    "skip_snapshots": 5,  # False or number of snapshots to skip
    "temporal_disaggregation": {
        "active": False,  # choose if temporally full complex dispatch optimization should be conducted
        "no_slices": 8,  # number of subproblems optimization is divided into
    },
    # Simplifications:
    "branch_capacity_factor": {"HV": 0.5, "eHV": 0.7},  # p.u. branch derating
    "load_shedding": True,  # meet the demand at value of loss load cost
    "foreign_lines": {
        "carrier": "AC",  # 'DC' for modeling foreign lines as links
        "capacity": "osmTGmod",  # 'osmTGmod', 'tyndp2020', 'ntc_acer' or 'thermal_acer'
    },
    "comments": None,
}

### Import the network

In [ ]:
#etrago = Etrago(args, json_path=None)
#etrago.build_network_from_db()

etrago.network.buses.carrier.value_counts()

In [ ]:
plot_carrier(etrago.network, carrier_links=["AC", "DC"])

In [ ]:
plot_carrier(etrago.network, carrier_links=["CH4"], carrier_buses=["CH4"])

### Cluster electrical network and attached technologies

In [ ]:
%%capture
#etrago.adjust_network()
#etrago.spatial_clustering()
#etrago.spatial_clustering_gas()
etrago2 = Etrago(csv_folder_name = "/home/carlos/Documents/Ego-n/etrago_presentation/before_lopf_60")

In [ ]:
plot_carrier(etrago2.network, carrier_links=["AC", "DC"])

### Cluster gas network and attached technologies

In [ ]:
plot_carrier(etrago2.network, carrier_links=["CH4"], carrier_buses=["CH4"])

### Reduce temporal complexity

In [ ]:
etrago2.network.snapshots

In [ ]:
etrago2.skip_snapshots()
etrago2.network.snapshots

### Run linear optimal power flow

In [ ]:
etrago2.lopf()

### Results

In [ ]:
etrago2.plot_grid(line_colors= "expansion_abs", bus_colors= "storage_expansion", bus_sizes= 0.000001)

In [ ]:
etrago2.calc_results()
etrago2.results